In [1]:
# dependencies
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import numpy as np
import getpass
import os
from datetime import datetime as dt
import glob

### Zillow Data Processing

In [2]:
listing_file = 'Resources/median_list_price.csv'
listing_df = pd.read_csv(listing_file, encoding='latin1')
listing_df.head()

,RegionName,City,State,Metro,CountyName,SizeRank,2015-01,2015-02,2015-03,2015-04,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1,299000.0,309000.0,325450.0,325000.0,...,399900.0,399900.0,399900.0,399900.0,399000.0,395000.0,399000.0,399950.0,399456.0,395000.0
1,Maryvale,Phoenix,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,2,115000.0,116900.0,118000.0,122900.0,...,186622.0,190000.0,192900.0,194500.0,193884.0,194900.0,197900.0,199000.0,199000.0,199900.0
2,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark County,3,179000.0,179999.0,185000.0,185000.0,...,272000.0,269893.5,270000.0,277700.0,275000.0,274949.5,274500.0,270000.0,270000.0,269500.0
3,Upper West Side,New York,NY,New York-Newark-Jersey City,New York County,4,NaN,NaN,NaN,NaN,...,1825000.0,1795000.0,1795000.0,1788000.0,1750000.0,1695000.0,1750000.0,1775000.0,1695000.0,1750000.0
4,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,5,315000.0,319250.0,315000.0,329000.0,...,475000.0,490000.0,499000.0,499000.0,498844.0,499000.0,512500.0,525000.0,528000.0,529000.0


In [3]:
port_listings = listing_df.loc[(listing_df['State']=='OR')]

port_listings2 = port_listings.loc[(port_listings['City']=='Portland')]

nbrhd_listings = port_listings2.drop(['State', 'Metro', 'CountyName', 'SizeRank'], axis=1)

final_nbrhd = nbrhd_listings.rename(columns={"RegionName": "Neighborhood"})

# listings_round = nbrhd_rename['2015-01', '2015-02','2015-03'].astype('int64')

final_nbrhd.fillna(0, inplace=True)

final_nbrhd

for col in final_nbrhd.columns[2:]:
    final_nbrhd[col] = final_nbrhd[col].astype('int64')

final_nbrhd

,Neighborhood,City,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
414,Powellhurst Gilbert,Portland,195000,199900,199900,212750,218500,218000,209950,224950,...,299000,299450,300000,309947,309473,309000,304900,299998,299947,299947
444,Hazelwood,Portland,191250,189950,207000,214950,209900,215000,214500,219700,...,312500,315000,309450,316357,309900,315463,324925,324900,312900,309950
474,Centennial,Portland,207500,199974,199900,207500,215975,215000,214999,224900,...,312450,309000,310000,310000,315000,324900,319900,319900,315450,314900
528,Lents,Portland,189950,179900,189450,206950,214900,229950,230000,235000,...,344950,339900,331500,329000,339000,334950,324900,320000,320000,335000
624,Montavilla,Portland,240000,270250,239800,259900,269949,267000,269000,269900,...,371750,385000,399000,399050,389900,378950,389900,389000,380000,379250
742,Northwest,Portland,0,0,0,0,0,0,0,0,...,432000,398000,430000,425000,389900,420000,399000,369400,410950,379000
779,St.Johns,Portland,231450,239900,262500,269000,255000,247450,255000,249950,...,357000,349900,359900,375000,359500,354500,369000,350000,359900,365000
802,Cully,Portland,0,0,0,0,0,0,0,0,...,384999,379400,422700,419900,395000,391950,412450,419500,419000,419000
842,Brentwood-Darlington,Portland,199999,200000,205000,219900,233055,249000,238250,235000,...,334750,340000,349893,345000,345000,334999,339725,345000,339900,335000
907,Richmond,Portland,0,0,0,483000,499000,489000,484450,464000,...,575000,565000,582000,549999,549999,515000,524900,529900,534900,525000


In [4]:
renting_file = 'Resources/zillow_neighborhood_rent.csv'
renting_df = pd.read_csv(renting_file, encoding='latin1')
renting_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2010-09,2010-10,2010-11,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,274772,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1,1070.05,1076.02,1088.45,...,1364.67,1361.97,1363.92,1368.71,1372.82,1375.39,1376.78,1379.89,1396.27,1403.48
1,112345,Maryvale,Phoenix,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,2,878.54,897.82,895.75,...,1163.25,1163.28,1166.00,1170.93,1179.64,1190.45,1201.33,1208.60,1224.04,1234.80
2,192689,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark County,3,1093.87,1095.46,1100.74,...,1303.40,1303.82,1307.68,1315.92,1324.92,1334.68,1344.17,1350.86,1370.02,1374.16
3,270958,Upper West Side,New York,NY,New York-Newark-Jersey City,New York County,4,3301.80,3340.97,3347.80,...,NaN,4107.81,4095.74,4086.67,4078.03,4070.69,4042.08,4066.10,4156.94,4205.98
4,118208,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,5,1781.15,1807.14,1804.85,...,2395.96,2391.67,2386.86,2387.62,2393.16,2400.53,2403.38,2399.27,2395.09,2405.24


In [5]:
renting_df = renting_df.loc[(renting_df['State']=='OR')]

renting_df = renting_df.loc[(renting_df['City']=='Portland')]

rental_listings = renting_df.drop(['State', 'Metro', 'CountyName', 'SizeRank', 'RegionID'], axis=1)

rental_rename = rental_listings.rename(columns={"RegionName": "Neighborhood"})

rental_rename

,Neighborhood,City,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
478,Powellhurst Gilbert,Portland,1225.67,NaN,1150.29,1124.15,NaN,1059.43,1032.82,1009.01,...,1538.71,1522.98,1506.70,1493.02,1485.12,1485.88,1495.84,1509.54,NaN,1590.57
512,Hazelwood,Portland,NaN,1199.13,NaN,1133.08,1108.12,1084.97,1069.87,1056.22,...,1481.01,1474.52,1463.86,1453.34,1445.49,1442.18,1445.47,1454.76,NaN,1556.95
545,Centennial,Portland,NaN,1173.41,1145.02,1125.68,1094.13,1072.88,1053.71,1028.25,...,1579.26,1568.53,1558.58,1548.53,1538.92,1524.36,1525.49,1538.96,NaN,1647.89
612,Lents,Portland,1167.86,1154.11,1137.36,1117.83,NaN,1052.04,NaN,NaN,...,1559.12,1549.46,1539.62,1533.38,1529.44,1529.78,1538.55,1546.39,NaN,NaN
739,Montavilla,Portland,1159.17,1188.83,1190.45,1179.39,NaN,NaN,NaN,1031.16,...,1615.09,1599.09,1586.26,1575.19,1569.56,1569.82,1573.68,1577.01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,Maplewood,Portland,1384.84,1412.88,1425.33,1432.01,1403.05,NaN,NaN,NaN,...,1727.50,1705.80,1685.18,1677.90,1673.88,1665.53,1662.99,1678.30,NaN,1782.06
5261,Hillside,Portland,1523.28,1554.27,1566.90,1542.74,1572.19,1572.14,1592.95,1608.43,...,2324.79,2314.29,2277.20,2253.70,2262.80,2258.71,2256.26,2276.50,2331.88,2275.46
5279,Arnold Creek,Portland,NaN,1755.64,1752.31,1720.58,1679.68,1631.70,1618.05,NaN,...,1916.52,1874.22,1854.92,1891.26,1931.35,NaN,NaN,1928.16,1961.53,1911.65
5408,Markham,Portland,1521.60,1548.03,1531.81,1525.59,1488.98,NaN,NaN,NaN,...,1686.31,1678.04,1667.77,1670.05,1668.02,1709.74,1731.52,1747.72,1758.79,1780.26


In [6]:
cols = rental_rename.columns[:2].tolist()
cols_df = cols.append(rental_rename.columns[54:].tolist())

In [7]:
cols = rental_rename.columns.tolist()

In [8]:
new_cols = cols[:2] + cols[54:]

In [9]:
new_rental = rental_rename[new_cols]

In [10]:
final_rental = new_rental.fillna(0)

for col in new_rental.columns[2:]:
    final_rental[col] = final_rental[col].astype('int64')

final_rental

,Neighborhood,City,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
478,Powellhurst Gilbert,Portland,1247,1250,1262,1276,1298,1320,1341,1361,...,1538,1522,1506,1493,1485,1485,1495,1509,0,1590
512,Hazelwood,Portland,1223,1220,1228,1240,1273,1303,1330,1354,...,1481,1474,1463,1453,1445,1442,1445,1454,0,1556
545,Centennial,Portland,1268,1271,1282,1291,1312,1340,1370,1394,...,1579,1568,1558,1548,1538,1524,1525,1538,0,1647
612,Lents,Portland,1256,1256,1266,1271,1297,1324,1352,1375,...,1559,1549,1539,1533,1529,1529,1538,1546,0,0
739,Montavilla,Portland,1327,1336,1352,1363,1388,1415,1450,1476,...,1615,1599,1586,1575,1569,1569,1573,1577,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,Maplewood,Portland,1408,1398,1426,1456,1494,1531,1563,1584,...,1727,1705,1685,1677,1673,1665,1662,1678,0,1782
5261,Hillside,Portland,2033,2037,2042,2079,2114,2170,2207,2241,...,2324,2314,2277,2253,2262,2258,2256,2276,2331,2275
5279,Arnold Creek,Portland,1662,1674,1713,0,0,0,1913,1963,...,1916,1874,1854,1891,1931,0,0,1928,1961,1911
5408,Markham,Portland,1396,1425,1466,0,0,1595,1585,1602,...,1686,1678,1667,1670,1668,1709,1731,1747,1758,1780


In [11]:
p = getpass.getpass(prompt="Password: ")
rds_connection_string = f"postgres:{p}@localhost:5432/airbnb_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

Password: ········


In [12]:
engine.table_names()

['rentals', 'listings']

In [12]:
final_nbrhd.to_sql(name='listings', con=engine, if_exists='replace', index=True)

In [13]:
final_rental.to_sql(name='rentals', con=engine, if_exists='replace', index=True)

In [14]:
pd.read_sql_query('select * from listings', con=engine).head()

,index,Neighborhood,City,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,414,Powellhurst Gilbert,Portland,195000,199900,199900,212750,218500,218000,209950,...,299000,299450,300000,309947,309473,309000,304900,299998,299947,299947
1,444,Hazelwood,Portland,191250,189950,207000,214950,209900,215000,214500,...,312500,315000,309450,316357,309900,315463,324925,324900,312900,309950
2,474,Centennial,Portland,207500,199974,199900,207500,215975,215000,214999,...,312450,309000,310000,310000,315000,324900,319900,319900,315450,314900
3,528,Lents,Portland,189950,179900,189450,206950,214900,229950,230000,...,344950,339900,331500,329000,339000,334950,324900,320000,320000,335000
4,624,Montavilla,Portland,240000,270250,239800,259900,269949,267000,269000,...,371750,385000,399000,399050,389900,378950,389900,389000,380000,379250


In [15]:
pd.read_sql_query('select * from rentals', con=engine).head()

,index,Neighborhood,City,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,478,Powellhurst Gilbert,Portland,1247,1250,1262,1276,1298,1320,1341,...,1538,1522,1506,1493,1485,1485,1495,1509,0,1590
1,512,Hazelwood,Portland,1223,1220,1228,1240,1273,1303,1330,...,1481,1474,1463,1453,1445,1442,1445,1454,0,1556
2,545,Centennial,Portland,1268,1271,1282,1291,1312,1340,1370,...,1579,1568,1558,1548,1538,1524,1525,1538,0,1647
3,612,Lents,Portland,1256,1256,1266,1271,1297,1324,1352,...,1559,1549,1539,1533,1529,1529,1538,1546,0,0
4,739,Montavilla,Portland,1327,1336,1352,1363,1388,1415,1450,...,1615,1599,1586,1575,1569,1569,1573,1577,0,0


### Airbnb Data Proessing

In [16]:
airbnb_files = []
for file in os.listdir("Resources"):
    if file.startswith("listing"):
        airbnb_files.append(file)
cols = ["date","id","name","neighbourhood","latitude","longitude","room_type","price","number_of_reviews","reviews_per_month"]

In [17]:
df = pd.DataFrame()
for file in airbnb_files:
    date = (file.split("(")[1].split(")")[0])
    temp = pd.read_csv("Resources/"+file)[cols[1:]]
    temp["date"] = dt.strptime(date, "%b %d %Y").strftime("%Y-%m-%d")
    df = pd.concat([df,temp[cols]])
    

In [18]:
df.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month
0,2016-04-05,689130,Exquisite private room w/sauna,Sullivan's Gulch,45.534838,-122.649000,Private room,100,54,1.25
1,2016-04-05,5794291,Private suites near it all- Room #6,Sullivan's Gulch,45.533373,-122.648236,Private room,100,34,3.71
2,2016-04-05,10885290,LOCATION NE : Queen Bunk Bed-TOP,Sullivan's Gulch,45.534529,-122.647962,Shared room,40,9,4.09
3,2016-04-05,4918034,Private suites near it all- Room #2,Sullivan's Gulch,45.533280,-122.647862,Private room,100,55,3.85
4,2016-04-05,5134256,Private suites near it all- Room #4,Sullivan's Gulch,45.533274,-122.646870,Private room,70,68,4.93


In [19]:
df.to_csv("data/All_airbnb_listings.csv",sep=",",index=False)

In [20]:
df.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month
0,2016-04-05,689130,Exquisite private room w/sauna,Sullivan's Gulch,45.534838,-122.649000,Private room,100,54,1.25
1,2016-04-05,5794291,Private suites near it all- Room #6,Sullivan's Gulch,45.533373,-122.648236,Private room,100,34,3.71
2,2016-04-05,10885290,LOCATION NE : Queen Bunk Bed-TOP,Sullivan's Gulch,45.534529,-122.647962,Shared room,40,9,4.09
3,2016-04-05,4918034,Private suites near it all- Room #2,Sullivan's Gulch,45.533280,-122.647862,Private room,100,55,3.85
4,2016-04-05,5134256,Private suites near it all- Room #4,Sullivan's Gulch,45.533274,-122.646870,Private room,70,68,4.93


In [21]:
sorted_airbnb = df.sort_values(
    ["id", "date"], ascending=True).reset_index(drop=True)
sorted_airbnb.head(15)

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month
0,2015-03-01,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00
1,2015-05-12,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00
2,2016-01-01,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.03
3,2017-10-04,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
4,2017-11-13,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
5,2017-12-09,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
6,2018-01-16,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
7,2018-02-08,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
8,2018-04-11,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
9,2018-05-13,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02


In [22]:
reviews = 0
current_id = 0
review_col = []
for i,row in sorted_airbnb.iterrows():
    if sorted_airbnb["id"].iloc[i] != current_id:
        current_id = sorted_airbnb["id"].iloc[i]
        reviews = sorted_airbnb["number_of_reviews"].iloc[i]
    else:
        reviews = sorted_airbnb["number_of_reviews"].iloc[i] - sorted_airbnb["number_of_reviews"].iloc[i-1]       
    
    review_col.append(reviews)

In [23]:
sorted_airbnb['reviews'] = review_col

In [24]:
sorted_airbnb.to_sql("airbnb_portland",con=engine,index=False, if_exists="replace")

In [25]:
engine.execute("SELECT * FROM airbnb_portland").fetchall()

[('2015-03-01', 7893, 'Alberta Arts 2brm', 'King', 45.561635987075434, -122.65572923241864, 'Entire home/apt', 95, 2, 0.0, 2),
 ('2015-05-12', 7893, 'Alberta Arts 2brm', 'King', 45.561635987075434, -122.65572923241864, 'Entire home/apt', 95, 2, 0.0, 0),
 ('2016-01-01', 7893, 'North Williams 2brm Home', 'Piedmont', 45.575478987075435, -122.66677723241864, 'Entire home/apt', 175, 2, 0.03, 0),
 ('2017-10-04', 7893, 'North Williams 2brm Home', 'Piedmont', 45.57547899, -122.66677720000001, 'Entire home/apt', 175, 2, 0.02, 0),
 ('2017-11-13', 7893, 'North Williams 2brm Home', 'Piedmont', 45.57547899, -122.66677720000001, 'Entire home/apt', 175, 2, 0.02, 0),
 ('2017-12-09', 7893, 'North Williams 2brm Home', 'Piedmont', 45.57547899, -122.66677720000001, 'Entire home/apt', 175, 2, 0.02, 0),
 ('2018-01-16', 7893, 'North Williams 2brm Home', 'Piedmont', 45.575478987075435, -122.66677723241864, 'Entire home/apt', 175, 2, 0.02, 0),
 ('2018-02-08', 7893, 'North Williams 2brm Home', 'Piedmont', 45.57